# using pyarrow to create a pa.Table of results

In [ ]:
import upath

In [ ]:
path_root = upath.UPath("gs://liulab/differential_composition_and_expression/20230310_21h01m15s")
# print(extras.get_parquet_paths(path_root))
# !gsutil ls "{path_root}/*"

## make pyarrow table

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.fs  # must be imported explicitly
import gcsfs

In [ ]:
# works
# - use fs from first UPath

paths = list(path_root.glob("**/gene_stats_*.parquet"))[:4]
dataset = ds.dataset(
    paths,
    format="parquet",
    filesystem=paths[0].fs,
)
print(dataset.filesystem)
dataset.head(5).to_pandas()

In [ ]:
# works
# - use fs from gcsfs.GCSFileSystem

paths = list(path_root.glob("**/gene_stats_*.parquet"))[:4]
print(paths)
dataset = ds.dataset(
    paths,
    format="parquet",
    filesystem=gcsfs.GCSFileSystem(),
)
print(dataset.filesystem)
dataset.head(5).to_pandas()

In [ ]:
# works
# - use str() to remove "gs://" (required for some reason)
# - use fs from pyarrow.fs.GcsFileSystem

paths = list(str(_)[5:] for _ in path_root.glob("**/gene_stats_*.parquet"))[:4]
print(paths)
dataset = ds.dataset(
    paths,
    format="parquet",
    filesystem=pyarrow.fs.GcsFileSystem(),  # required
)
print(dataset.filesystem)
dataset.head(5).to_pandas()

### include path

In [ ]:
print(next(path_root.glob("**/gene_stats_*.parquet")))

In [ ]:
paths = list(path_root.glob("**/gene_stats_*.parquet"))
dataset = ds.dataset(
    paths,
    format="parquet",
    filesystem=gcsfs.GCSFileSystem(),  # required
    partitioning="hive",
)

# table_gene_stats = dataset.to_table()

# projection = dataset.schema.names + ["__filename"]
# table_gene_stats = dataset.to_table(columns=projection)

columns = dataset.schema.names + ["__filename"]
scanner = dataset.scanner(columns=columns)
table_gene_stats = scanner.head(100)

# rename column
# table_gene_stats = table_gene_stats.rename_columns(
#     table_gene_stats.column_names[:-1] + ["path"]
# )
# table_gene_stats.to_pandas()

In [ ]:
table_gene_stats.to_pandas()

In [ ]:
# table_gene_stats = dataset.to_table()